<a href="https://colab.research.google.com/github/jeron-williams/Easy_Visa_Classification_Hypertuning_Bagging_Boosting/blob/main/Copy_of_EasyVisa_Full_Code_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

Business communities in the United States are facing high demand for human resources, but one of the constant challenges is identifying and attracting the right talent, which is perhaps the most important element in remaining competitive. Companies in the United States look for hard-working, talented, and qualified individuals both locally as well as abroad.

The Immigration and Nationality Act (INA) of the US permits foreign workers to come to the United States to work on either a temporary or permanent basis. The act also protects US workers against adverse impacts on their wages or working conditions by ensuring US employers' compliance with statutory requirements when they hire foreign workers to fill workforce shortages. The immigration programs are administered by the Office of Foreign Labor Certification (OFLC).

OFLC processes job certification applications for employers seeking to bring foreign workers into the United States and grants certifications in those cases where employers can demonstrate that there are not sufficient US workers available to perform the work at wages that meet or exceed the wage paid for the occupation in the area of intended employment.

### Objective

In FY 2016, the OFLC processed 775,979 employer applications for 1,699,957 positions for temporary and permanent labor certifications. This was a nine percent increase in the overall number of processed applications from the previous year. The process of reviewing every case is becoming a tedious task as the number of applicants is increasing every year.

The increasing number of applicants every year calls for a Machine Learning based solution that can help in shortlisting the candidates having higher chances of VISA approval. OFLC has hired the firm EasyVisa for data-driven solutions. You as a data  scientist at EasyVisa have to analyze the data provided and, with the help of a classification model:

* Facilitate the process of visa approvals.
* Recommend a suitable profile for the applicants for whom the visa should be certified or denied based on the drivers that significantly influence the case status.

### Data Description

The data contains the different attributes of employee and the employer. The detailed data dictionary is given below.

* case_id: ID of each visa application
* continent: Information of continent the employee
* education_of_employee: Information of education of the employee
* has_job_experience: Does the employee has any job experience? Y= Yes; N = No
* requires_job_training: Does the employee require any job training? Y = Yes; N = No
* no_of_employees: Number of employees in the employer's company
* yr_of_estab: Year in which the employer's company was established
* region_of_employment: Information of foreign worker's intended region of employment in the US.
* prevailing_wage:  Average wage paid to similarly employed workers in a specific occupation in the area of intended employment. The purpose of the prevailing wage is to ensure that the foreign worker is not underpaid compared to other workers offering the same or similar service in the same area of employment.
* unit_of_wage: Unit of prevailing wage. Values include Hourly, Weekly, Monthly, and Yearly.
* full_time_position: Is the position of work full-time? Y = Full Time Position; N = Part Time Position
* case_status:  Flag indicating if the Visa was certified or denied

## Installing and Importing the necessary libraries

**Note**: *After running the above cell, kindly restart the notebook kernel and run all cells sequentially from the below.*

In [ ]:
# Standard library utils
import time
import warnings

# Data wrangling
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Model selection / CV
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,    # preferred for classification
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
)

# Metrics
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    make_scorer,
)

from sklearn.compose import ColumnTransformer
from collections import Counter

# Preprocessing
from sklearn.preprocessing import OneHotEncoder

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# XGBoost (optional)
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# Global config
warnings.filterwarnings("ignore")
sns.set(style="whitegrid", context="notebook")


## Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data= pd.read_csv("/content/drive/My Drive/Advanced Machine Learning Module 3/EasyVisa.csv")

copydata = data.copy()

In [ ]:
# Convert notebook -> HTML in the same Drive folder
!jupyter nbconvert --to html "/content/drive/MyDrive/Advanced Machine Learning Module 3/EasyVisa_Full_Code_Notebook.ipynb"

# Download the HTML that was just created
from google.colab import files
files.download("/content/drive/MyDrive/Advanced Machine Learning Module 3/EasyVisa_Full_Code_Notebook.html")


## Overview of the Dataset

#### View the first and last 5 rows of the dataset

In [ ]:
# First 5 rows (default)
copydata.head()

#### Understand the shape of the dataset

In [ ]:
# (rows, columns)
copydata.shape

#### Check the data types of the columns for the dataset

In [ ]:
# Schema summary: columns, non-nulls, dtypes, memory
copydata.info()

In [ ]:
for feature in copydata.columns: # Loop through all columns in the dataframe
    if copydata[feature].dtype == 'object': # Only apply for columns with categorical strings
        copydata[feature] = pd.Categorical(copydata[feature])# Replace strings with an integer
copydata.head(10)

In [ ]:
# Column data types
print(copydata.dtypes)

In [ ]:
# Missing values per column (counts)
copydata.isnull().sum()

In [ ]:
copydata.duplicated().sum()

## Exploratory Data Analysis (EDA)

#### Let's check the statistical summary of the data

In [ ]:
# Descriptive stats for all dtypes (numeric, object, category, datetime)
copydata.describe(include='all')

#### Fixing the negative values in number of employees columns

In [ ]:
# Count negatives and list offending rows
print("Number of negatives:", (copydata['no_of_employees'] < 0).sum())

negatives = copydata[copydata['no_of_employees'] < 0]
print(negatives)

In [ ]:
# Make values non-negative, then verify none remain negative
copydata['no_of_employees'] = copydata['no_of_employees'].abs()
print("Number of negatives:", (copydata['no_of_employees'] < 0).sum())

#### Let's check the count of each unique category in each of the categorical variables

In [ ]:
# Distinct values per column (NaN excluded)
copydata.nunique()

In [ ]:
# Descriptive stats for all dtypes
print(copydata.describe(include='all'))

### Univariate Analysis

In [ ]:
def histogram_boxplot(data, feature, figsize=(15, 10), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (15,10))
    kde: whether to show the density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a triangle will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

    plt.show()

In [ ]:
# function to create labeled barplots


def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 1, 5))
    else:
        plt.figure(figsize=(n + 1, 5))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

In [ ]:
# Employees: distribution + outliers
histogram_boxplot(copydata, 'no_of_employees')

# Year established: distribution + outliers
histogram_boxplot(copydata, 'yr_of_estab')

# Prevailing wage: distribution + outliers (often right-skewed)
histogram_boxplot(copydata, 'prevailing_wage')



In [ ]:
# Labeled bar charts; perc=True shows percentages

# Continent distribution (%)
labeled_barplot(copydata, 'continent', perc=True)

# Education level of employee (%)
labeled_barplot(copydata, 'education_of_employee', perc=True)

# Has prior job experience (% yes/no)
labeled_barplot(copydata, 'has_job_experience', perc=True)

# Requires job training (% yes/no)
labeled_barplot(copydata, 'requires_job_training', perc=True)

# Number of employees (%); bin numeric if many uniques
labeled_barplot(copydata, 'no_of_employees', perc=True)

# Region of employment (%)
labeled_barplot(copydata, 'region_of_employment', perc=True)

# Prevailing wage (%); bin/quantile if many uniques
labeled_barplot(copydata, 'prevailing_wage', perc=True)

# Unit of wage (%)
labeled_barplot(copydata, 'unit_of_wage', perc=True)

# Full-time vs part-time (%)
labeled_barplot(copydata, 'full_time_position', perc=True)



In [ ]:
labeled_barplot(copydata, 'case_status', perc=True)

#### Observations on education of employee

* The highest education in this dataset is a bachelor's degree at 40.2%.
* The next highest education is 37.8% at the master's degree level.
* 78% of the work visa applications come from people with either a bachelor's degree or a master's degree.
* 13.4% of the applicants have a highschool level education
* 8.6% have a doctorate level education

#### Observations on region of employment

* The northeast region has the highest percentage of applicants for work visas at 28.2%
* The south region is 27.5%
* The west is 25.8%
* This could owe to the coastsal closeness of these areas as well as other features like the types of work that require applicants outside the US being more predominant in these regions.
* Midwest is 16.9%
* Island is 1.5%

#### Observations on job experience

* 58.1% of the applicants have job experience
* 41.9% of the job applicants do not have job experience.
* This could be due to the vast majority of applicants without job experience being recent college graduates.
* The high level of job experience could be individuals searching for better opportunities or from pursuing extended education and working during the process.

#### Observations on case status

* 66.8% of the applicants are certified
* 33.2% of the applicants are denied
* 2/3 of those who apply are certified for work visas.

### Bivariate Analysis

**Creating functions that will help us with further analysis.**

In [ ]:
### function to plot distributions wrt target


def distribution_plot_wrt_target(data, predictor, target):

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    target_uniq = data[target].unique()

    axs[0, 0].set_title("Distribution of target for target=" + str(target_uniq[0]))
    sns.histplot(
        data=data[data[target] == target_uniq[0]],
        x=predictor,
        kde=True,
        ax=axs[0, 0],
        color="teal",
        stat="density",
    )

    axs[0, 1].set_title("Distribution of target for target=" + str(target_uniq[1]))
    sns.histplot(
        data=data[data[target] == target_uniq[1]],
        x=predictor,
        kde=True,
        ax=axs[0, 1],
        color="orange",
        stat="density",
    )

    axs[1, 0].set_title("Boxplot w.r.t target")
    sns.boxplot(data=data, x=target, y=predictor, ax=axs[1, 0], palette="gist_rainbow")

    axs[1, 1].set_title("Boxplot (without outliers) w.r.t target")
    sns.boxplot(
        data=data,
        x=target,
        y=predictor,
        ax=axs[1, 1],
        showfliers=False,
        palette="gist_rainbow",
    )

    plt.tight_layout()
    plt.show()

In [ ]:
def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 5, 5))
    plt.legend(
        loc="lower left", frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

In [ ]:
# Class-wise distribution: employees vs case_status
distribution_plot_wrt_target(copydata, 'no_of_employees', 'case_status')

# Class-wise distribution: year established vs case_status
distribution_plot_wrt_target(copydata, 'yr_of_estab', 'case_status')

# Class-wise distribution: prevailing wage vs case_status
distribution_plot_wrt_target(copydata, 'prevailing_wage', 'case_status')


In [ ]:
# Stacked counts by case_status: continent
stacked_barplot(copydata, 'continent', 'case_status')

# Stacked counts by case_status: education
stacked_barplot(copydata, 'education_of_employee', 'case_status')

# Stacked counts by case_status: prior job experience
stacked_barplot(copydata, 'has_job_experience', 'case_status')

# Stacked counts by case_status: requires training
stacked_barplot(copydata, 'requires_job_training', 'case_status')

# Stacked counts by case_status: employees (bin if many uniques)
stacked_barplot(copydata, 'no_of_employees', 'case_status')

# Stacked counts by case_status: region
stacked_barplot(copydata, 'region_of_employment', 'case_status')

# Stacked counts by case_status: wage (bin/quantile for readability)
stacked_barplot(copydata, 'prevailing_wage', 'case_status')

# Stacked counts by case_status: wage unit
stacked_barplot(copydata, 'unit_of_wage', 'case_status')

# Stacked counts by case_status: full-time vs part-time
stacked_barplot(copydata, 'full_time_position', 'case_status')


In [ ]:
# 1) Keep only numeric columns
num = copydata.select_dtypes(include=[np.number]).copy()

# 2) Optional hygiene: drop mostly-empty and constant columns
num = num.dropna(axis=1, thresh=int(0.8 * len(num)))                 # ≥80% non-null
const_cols = [c for c in num.columns if num[c].nunique(dropna=True) <= 1]
num = num.drop(columns=const_cols)

if num.empty:
    raise ValueError("No numeric columns available after filtering.")

# 3) Correlation + heatmap (lower triangle)
corr = num.corr(numeric_only=True)

plt.figure(figsize=(12, 8))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f",
            cmap="coolwarm", vmin=-1, vmax=1, cbar_kws={'shrink': .8})
plt.title("Correlation Heatmap (numeric columns only)")
plt.tight_layout()
plt.show()


In [ ]:
# Average prevailing wage by region (ascending)
grouped_roe_pw = (
    copydata.groupby('region_of_employment')['prevailing_wage']
            .mean()
            .sort_values()
)
print(grouped_roe_pw)


In [ ]:
# Mean wage per case_status (simple groupby)
print(copydata.groupby('case_status')['prevailing_wage'].mean())

# Convert to numeric (coerce errors → NaN)
copydata['pw'] = pd.to_numeric(copydata['prevailing_wage'], errors='coerce')

# Drop rows missing pw or case_status
coydata = copydata.dropna(subset=['pw', 'case_status'])

# IQR filter: keep rows within 1.5×IQR per group
def iqr_filter(g):
    q1, q3 = g['pw'].quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    return g[(g['pw'] >= low) & (g['pw'] <= high)]

# Apply per case_status, then recompute means
copydata_iqr = copydata.groupby('case_status', group_keys=False).apply(iqr_filter)
means_iqr = copydata_iqr.groupby('case_status')['pw'].mean()
print(means_iqr.round(2))


#### Does higher education increase the chances of visa certification for well-paid jobs abroad?

* Having higher education increases the likelihood of receiving a work visa in a linear fashion moving from highschool education to doctorate level.
* About 37% of those that applied with a highschool education received work visas
* A little over 60% of those that applied, with a bachelor's degree, received a work visa
* Right at 80% of those that applied for a work visa with a master's degree received a work visa.
* Right at 90% of those that applied for a work visa with a doctorate were certified.


#### How does visa status vary across different continents?

* Visa status progresses in a linear fashion across contnenents in the following order. South America, North America, Ocenaia, Asia, Africa, and Europe.
* South America has just below a 60% certification rate.
* North America has just above a 60% approval rate.
* Oceania has about a 63% certification rate
* Asia has about a 65% certification rate.
* Africa is around a 70% certification rate.
* Europe has around an 80% certification rate.
* Different continents see lower and higher rates of certification, by virtue of the continent alone.

#### Does having prior work experience influence the chances of visa certification for career opportunities abroad?

* Those with job experience have just under an 80% chance of certification, while those without job experience have just under a 60% chance of certification.

#### Is the prevailing wage consistent across all regions of the US?

* The prevailing wage is fairly consistent between the south, northeast, and west regions as far as an average is concerned. The prevailing wage average differs by around 6000 dollars.
* Island and Midwest regions are also very similar, as far as an average is concerned. There average variance is only 200 dollars.
* The average prevailing wage between all regions has some significant variance.

#### Does visa status vary with changes in the prevailing wage set to protect both local talent and foreign workers?

* The prevailing wage of those certified is an average of 10,000 dollars difference than those who are not certified. This indicates that the prevailing wage is protectiing both the foerign and US workers.
* After dropping all outliers similar trends in the prevailing wage result. The difference is about 7000 dollars.
* Either way those certified receieve 7-10 thousand dollars more in the prevailing wage than those who are not certified. This indicates a correct relationship with the prevailing wage and certification.
* The prevailing wage does differ between the certified and the not certified. It does this in a correct way where those certified have a higher prevailing wage than those not certified.

#### Does the unit of prevailing wage (Hourly, Weekly, etc.) have any impact on the likelihood of visa application certification?

* Houlry wage receives about a 37% approval rate.
* Weekly and monthly receive around a 60% approval rate.
* Yearly receives around a 70% approval rate.
* In some way it seems that the unit does have correlation with certification verse non-certification.

## Data Pre-processing

### Outlier Check

In [ ]:
def mark_outliers_iqr(g, col='pw', k=1.5, add_bounds=False):
    """
    Add a boolean flag for IQR outliers within a group.

    g : DataFrame (one group from groupby)
    col : column name to check (e.g., 'pw')
    k : fence multiplier (1.5 = standard; 3.0 = 'extreme' outliers)
    add_bounds : if True, also add q1/q3/low/high columns for reference
    """
    s = pd.to_numeric(g[col], errors='coerce')
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1

    # Handle degenerate groups (iqr == 0 or all NaN)
    if pd.isna(iqr) or iqr == 0:
        out = g.copy()
        out[f'{col}_outlier_iqr'] = False
        if add_bounds:
            out[f'{col}_q1'], out[f'{col}_q3']   = q1, q3
            out[f'{col}_low'], out[f'{col}_high'] = q1, q3
        return out

    low, high = q1 - k*iqr, q3 + k*iqr
    out = g.copy()
    out[f'{col}_outlier_iqr'] = (s < low) | (s > high)
    if add_bounds:
        out[f'{col}_q1'], out[f'{col}_q3']   = q1, q3
        out[f'{col}_low'], out[f'{col}_high'] = low, high
    return out



In [ ]:
# Work on a copy of the dataset to avoid altering the original
df = copydata.copy()

# Convert 'prevailing_wage' to numeric
# - Non-numeric entries become NaN (via errors='coerce')
# - Stores in a new column 'pw'
df['pw'] = pd.to_numeric(df['prevailing_wage'], errors='coerce')


In [ ]:
# Flag outliers in 'pw' using IQR (k=1.5, keep bounds too)
df_marked = mark_outliers_iqr(df, col='pw', k=1.5, add_bounds=True)

# Separate into outliers and inliers
outliers = df_marked[df_marked['pw_outlier_iqr']]
inliers  = df_marked[~df_marked['pw_outlier_iqr']]

# Shapes (rows, cols) of each set
print(outliers.shape)
print(inliers.shape)

# Total rows across both sets
total_rows = outliers.shape[0] + inliers.shape[0]

# Outlier and inlier counts + percentages
print(f"outliers rows: {outliers.shape[0]:,} / {total_rows:,} ({outliers.shape[0]/total_rows:.2%})")
print(f"inliers  rows: {inliers.shape[0]:,} / {total_rows:,} ({inliers.shape[0]/total_rows:.2%})")

# Notes:
# - df_marked keeps original data + flag column ('pw_outlier_iqr') and IQR bounds.
# - Shapes confirm split sizes; percentages check outlier prevalence.


### Data Preparation for modeling

In [ ]:
# 0) Split target and features
X = data.drop(columns=['case_status'])
y = data['case_status']

# 1) Make binary target (optional: set minority as positive=1)
minority = y.value_counts().idxmin()
y_bin = (y == minority).astype(int)

# 2) Train / Test split (e.g., 80/20)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y_bin, test_size=0.20, random_state=1, stratify=y_bin
)

# 3) From remaining 80%, carve out Validation (e.g., 20% of original → 25% of remaining)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=1, stratify=y_train_full
)

# 4) Identify categorical columns from TRAIN ONLY
cat_cols = X_train.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

# If you have numeric-coded categoricals, declare them and cast:
# num_as_cats = ['colA', 'colB']
# for df in (X_train, X_val, X_test):
#     df[num_as_cats] = df[num_as_cats].astype('category')
# cat_cols = sorted(set(cat_cols).union(num_as_cats))

# 5) One-hot encode each split separately, then align to TRAIN columns
X_train_d = pd.get_dummies(X_train, columns=cat_cols, drop_first=True, dtype='uint8')
X_val_d   = pd.get_dummies(X_val,   columns=cat_cols, drop_first=True, dtype='uint8')
X_test_d  = pd.get_dummies(X_test,  columns=cat_cols, drop_first=True, dtype='uint8')

# Align val/test to train’s columns (drop unknowns, add missing as 0)
X_val_d  = X_val_d.reindex(columns=X_train_d.columns, fill_value=0)
X_test_d = X_test_d.reindex(columns=X_train_d.columns, fill_value=0)

# 6) Quick sanity checks
print("Train:", X_train_d.shape, "Val:", X_val_d.shape, "Test:", X_test_d.shape)
print("y train/val/test:", y_train.shape, y_val.shape, y_test.shape)
print("Class balance (train):\n", y_train.value_counts(normalize=True))
print("Class balance (val):\n",   y_val.value_counts(normalize=True))
print("Class balance (test):\n",  y_test.value_counts(normalize=True))


## Model Building

### Model Evaluation Criterion

- Choose the primary metric to evaluate the model on
- Elaborate on the rationale behind choosing the metric
- Xgboost is going to likely be the ideal model

First, let's create functions to calculate different metrics and confusion matrix so that we don't have to use the same code repeatedly for each model.
* The `model_performance_classification_sklearn` function will be used to check the model performance of models.
* The `confusion_matrix_sklearn` function will be used to plot the confusion matrix.

In [ ]:
# defining a function to compute different metrics to check performance of a classification model built using sklearn


def model_performance_classification_sklearn(model, predictors, target):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred)  # to compute Recall
    precision = precision_score(target, pred)  # to compute Precision
    f1 = f1_score(target, pred)  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1": f1,},
        index=[0],
    )

    return df_perf

In [ ]:
def confusion_matrix_sklearn(model, predictors, target):
    """
    To plot the confusion_matrix with percentages

    model: classifier
    predictors: independent variables
    target: dependent variable
    """
    y_pred = model.predict(predictors)
    cm = confusion_matrix(target, y_pred)
    labels = np.asarray(
        [
            ["{0:0.0f}".format(item) + "\n{0:.2%}".format(item / cm.flatten().sum())]
            for item in cm.flatten()
        ]
    ).reshape(2, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=labels, fmt="")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

#### Defining scorer to be used for cross-validation and hyperparameter tuning

* Both false positives and false negatives need to be avoided. For this reason the f1 score is the most ideal metric for this situation.

**We are now done with pre-processing and evaluation criterion, so let's start building the model.**

### Model building with Original data

In [ ]:
# 1) Train Bagging with defaults (no hyperparameter tuning)
bag = BaggingClassifier(random_state=1)   # defaults: base estimator = DecisionTreeClassifier(), n_estimators=10
bag.fit(X_train_d, y_train)

In [ ]:
# Train-set performance
bagging_estimator_score_train = model_performance_classification_sklearn(
    bag, X_train_d, y_train
)
bagging_estimator_score_train


In [ ]:
# Validation-set performance
bagging_estimator_score_val = model_performance_classification_sklearn(
    bag, X_val_d, y_val
)
bagging_estimator_score_val


In [ ]:
bagging_estimator_cm = confusion_matrix_sklearn(bag, X_val_d, y_val)
bagging_estimator_cm

In [ ]:
dt = DecisionTreeClassifier(random_state=1)
dt.fit(X_train_d, y_train)

In [ ]:
dt_estimator_score_train = model_performance_classification_sklearn(
    dt, X_train_d, y_train
)
dt_estimator_score_train

In [ ]:
dt_estimator_score_val = model_performance_classification_sklearn(
    dt, X_val_d, y_val
)
dt_estimator_score_val

In [ ]:
dt_estimator_cm = confusion_matrix_sklearn(dt, X_val_d, y_val)
dt_estimator_cm

In [ ]:
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train_d, y_train)

In [ ]:
rf_estimator_score_train = model_performance_classification_sklearn(
    rf, X_train_d, y_train
)
rf_estimator_score_train

In [ ]:
rf_estimator_score_val = model_performance_classification_sklearn(
    rf, X_val_d, y_val
)
rf_estimator_score_val

In [ ]:
rf_estimator_cm = confusion_matrix_sklearn(rf, X_val_d, y_val)
rf_estimator_cm

In [ ]:
abc = AdaBoostClassifier(random_state=1)
abc.fit(X_train_d, y_train)

In [ ]:
abc_estimator_score_train = model_performance_classification_sklearn(
    abc, X_train_d, y_train
)
abc_estimator_score_train

In [ ]:
abc_estimator_score_val = model_performance_classification_sklearn(
    abc, X_val_d, y_val
)
abc_estimator_score_val

In [ ]:
abc_estimator_cm = confusion_matrix_sklearn(abc, X_val_d, y_val)
abc_estimator_cm

In [ ]:
xgb= XGBClassifier(random_state=1)
xgb.fit(X_train_d,y_train)

In [ ]:
xgb_estimator_score_train= model_performance_classification_sklearn(xgb,X_train_d, y_train,)
xgb_estimator_score_train

In [ ]:
xgb_estimator_score_val= model_performance_classification_sklearn(xgb,X_val_d, y_val,)
xgb_estimator_score_val

In [ ]:
xgb_estimator_cm = confusion_matrix_sklearn(xgb, X_val_d, y_val)
xgb_estimator_cm

In [ ]:
gb = GradientBoostingClassifier(random_state=1)
gb.fit(X_train_d, y_train)

In [ ]:
gb_estimator_score_train = model_performance_classification_sklearn(
    gb, X_train_d, y_train
)
gb_estimator_score_train

In [ ]:
gb_estimator_score_val = model_performance_classification_sklearn(
    gb, X_val_d, y_val
)
gb_estimator_score_val

In [ ]:
gb_estimator_cm = confusion_matrix_sklearn(gb, X_val_d, y_val)
gb_estimator_cm

* Gradient boosting and adaboost both generalized well. But they were both underfit with poor f1 scores, both under 60.
* Every other model was overfit and also performed poorly with the f1 scores on validation sets.

### Model Building with Oversampled data

In [ ]:
# Ensure y is 1-D
y_train_1d = np.asarray(y_train).ravel()

def counts(y):
    return pd.Series(y).value_counts(dropna=False)

print("Before OverSampling (value_counts):")
print(counts(y_train_1d), "\n")

# OPTIONAL: choose k_neighbors safely if the minority class is tiny
minority_count = counts(y_train_1d).min()
k = max(1, min(5, minority_count - 1))  # keep k < minority_count

sm = SMOTE(sampling_strategy=1.0, k_neighbors=k, random_state=1)

# IMPORTANT: use the one-hot encoded training matrix
X_train_over, y_train_over = sm.fit_resample(X_train_d, y_train_1d)

print("After OverSampling (value_counts):")
print(counts(y_train_over), "\n")

print("After OverSampling, the shape of train_X:", X_train_over.shape)
print("After OverSampling, the shape of train_y:", y_train_over.shape, "\n")

# If you specifically want counts for label 0 and 1 (only if labels truly are 0/1):
def lbl_count(y, lbl): return int(np.sum(np.asarray(y).ravel() == lbl))
print("Before OverSampling, count of label '1':", lbl_count(y_train_1d, 1))
print("Before OverSampling, count of label '0':", lbl_count(y_train_1d, 0))
print("After OverSampling, count of label '1':", lbl_count(y_train_over, 1))
print("After OverSampling, count of label '0':", lbl_count(y_train_over, 0))





In [ ]:
bag_over = BaggingClassifier(random_state=1)
bag_over.fit(X_train_over, y_train_over)

In [ ]:
bagging_estimator_score_train_over= model_performance_classification_sklearn(bag_over,X_train_over, y_train_over,)
bagging_estimator_score_train_over

In [ ]:
bagging_estimator_score_val_over= model_performance_classification_sklearn(bag_over,X_val_d, y_val,)
bagging_estimator_score_val_over

In [ ]:
bagging_estimator_cm_over = confusion_matrix_sklearn(bag_over, X_val_d, y_val)
bagging_estimator_cm_over

In [ ]:
dt_over = DecisionTreeClassifier(random_state=1)
dt_over.fit(X_train_over, y_train_over)

In [ ]:
dt_estimator_score_train_over= model_performance_classification_sklearn(dt_over,X_train_over, y_train_over,)
dt_estimator_score_train_over

In [ ]:
dt_estimator_score_val_over= model_performance_classification_sklearn(dt_over,X_val_d, y_val,)
dt_estimator_score_val_over

In [ ]:
dt_estimator_cm_over = confusion_matrix_sklearn(dt_over, X_val_d, y_val)
dt_estimator_cm_over

In [ ]:
rf_over = RandomForestClassifier(random_state=1)
rf_over.fit(X_train_over, y_train_over)

In [ ]:
rf_estimator_score_train_over= model_performance_classification_sklearn(rf_over,X_train_over, y_train_over,)
rf_estimator_score_train_over

In [ ]:
rf_estimator_score_val_over= model_performance_classification_sklearn(rf_over,X_val_d, y_val,)
rf_estimator_score_val_over

In [ ]:
rf_estimator_cm_over = confusion_matrix_sklearn(rf_over, X_val_d, y_val)
rf_estimator_cm_over

In [ ]:
abc_over = AdaBoostClassifier(random_state=1)
abc_over.fit(X_train_over, y_train_over)

In [ ]:
abc_estimator_score_train_over= model_performance_classification_sklearn(abc_over,X_train_over, y_train_over,)
abc_estimator_score_train_over

In [ ]:
abc_estimator_score_val_over= model_performance_classification_sklearn(abc_over,X_val_d, y_val,)
abc_estimator_score_val_over

In [ ]:
abc_estimator_cm_over = confusion_matrix_sklearn(abc_over, X_val_d, y_val)
abc_estimator_cm_over

In [ ]:
xgb_estimator_over=XGBClassifier(random_state=1)
xgb_estimator_over.fit(X_train_over,y_train_over)

In [ ]:
xgb_estimator_over_score_train= model_performance_classification_sklearn(xgb_estimator_over,X_train_over, y_train_over,)
xgb_estimator_over_score_train

In [ ]:
xgb_estimator_over_score_val= model_performance_classification_sklearn(xgb_estimator_over,X_val_d, y_val,)
xgb_estimator_over_score_val

In [ ]:
xgb_estimator_over_cm = confusion_matrix_sklearn(xgb_estimator_over, X_val_d, y_val)
xgb_estimator_over_cm

In [ ]:
gb_over = GradientBoostingClassifier(random_state=1)
gb_over.fit(X_train_over, y_train_over)

In [ ]:
gb_estimator_score_train_over= model_performance_classification_sklearn(gb_over,X_train_over, y_train_over,)
gb_estimator_score_train_over

In [ ]:
gb_estimator_score_val_over= model_performance_classification_sklearn(gb_over,X_val_d, y_val,)
gb_estimator_score_val_over

In [ ]:
gb_estimator_cm_over = confusion_matrix_sklearn(gb_over, X_val_d, y_val)
gb_estimator_cm_over

* All oversampled data models were overfit and did not generalize well

### Model Building with Undersampled data

In [ ]:
# Make sure y is 1-D
y_train_1d = np.asarray(y_train).ravel()

print("Before Under Sampling (value_counts):")
print(pd.Series(y_train_1d).value_counts(), "\n")

# Perform random undersampling on the ONE-HOT TRAIN set
rus = RandomUnderSampler(random_state=1, sampling_strategy=1.0)  # 1.0 => balance classes 1:1
X_train_un, y_train_un = rus.fit_resample(X_train_d, y_train_1d)  # <-- use X_train_d

print("After Under Sampling (value_counts):")
print(pd.Series(y_train_un).value_counts(), "\n")

print("After Under Sampling, the shape of train_X:", X_train_un.shape)
print("After Under Sampling, the shape of train_y:", y_train_un.shape, "\n")

# OPTIONAL: if your labels are actually 0/1 and you want explicit counts
if set(np.unique(y_train_1d)) == {0, 1}:
    print("Before Under Sampling, count of label '1':", np.sum(y_train_1d == 1))
    print("Before Under Sampling, count of label '0':", np.sum(y_train_1d == 0), "\n")
    print("After Under Sampling, count of label '1':", np.sum(y_train_un == 1))
    print("After Under Sampling, count of label '0':", np.sum(y_train_un == 0), "\n")



In [ ]:
bag_under = BaggingClassifier(random_state=1)
bag_under.fit(X_train_un, y_train_un)

In [ ]:
bagging_estimator_under_score_train= model_performance_classification_sklearn(bag_under,X_train_un, y_train_un,)
bagging_estimator_under_score_train

In [ ]:
bagging_estimator_under_score_val= model_performance_classification_sklearn(bag_under,X_val_d, y_val,)
bagging_estimator_under_score_val

In [ ]:
bagging_estimator_under_cm = confusion_matrix_sklearn(bag_under, X_val_d, y_val)
bagging_estimator_under_cm

In [ ]:
dt_under = DecisionTreeClassifier(random_state=1)
dt_under.fit(X_train_un, y_train_un)

In [ ]:
dt_under_score_train= model_performance_classification_sklearn(dt_under,X_train_un, y_train_un,)
dt_under_score_train

In [ ]:
dt_under_score_val= model_performance_classification_sklearn(dt_under,X_val_d, y_val,)
dt_under_score_val

In [ ]:
dt_under_cm = confusion_matrix_sklearn(dt_under, X_val_d, y_val)
dt_under_cm

In [ ]:
rf_under = RandomForestClassifier(random_state=1)
rf_under.fit(X_train_un, y_train_un)

In [ ]:
rf_under_score_train= model_performance_classification_sklearn(rf_under,X_train_un, y_train_un,)
rf_under_score_train

In [ ]:
rf_under_score_val= model_performance_classification_sklearn(rf_under,X_val_d, y_val,)
rf_under_score_val

In [ ]:
rf_under_cm = confusion_matrix_sklearn(rf_under, X_val_d, y_val)
rf_under_cm

In [ ]:
abc_under = AdaBoostClassifier(random_state=1)
abc_under.fit(X_train_un, y_train_un)

In [ ]:
abc_under_score_train= model_performance_classification_sklearn(abc_under,X_train_un, y_train_un,)
abc_under_score_train

In [ ]:
abc_under_score_val= model_performance_classification_sklearn(abc_under,X_val_d, y_val,)
abc_under_score_val

In [ ]:
abc_under_cm = confusion_matrix_sklearn(abc_under, X_val_d, y_val)
abc_under_cm

In [ ]:
xgb_estimator_under=XGBClassifier(random_state=1)
xgb_estimator_under.fit(X_train_un,y_train_un)

In [ ]:
xgb_estimator_under_score_train= model_performance_classification_sklearn(xgb_estimator_under,X_train_un, y_train_un,)
xgb_estimator_under_score_train

In [ ]:
xgb_estimator_under_score_val= model_performance_classification_sklearn(xgb_estimator_under,X_val_d, y_val,)
xgb_estimator_under_score_val

In [ ]:
xgb_estimator_under_cm = confusion_matrix_sklearn(xgb_estimator_under, X_val_d, y_val)
xgb_estimator_under_cm

In [ ]:
gb_under = GradientBoostingClassifier(random_state=1)
gb_under.fit(X_train_un, y_train_un)

In [ ]:
gb_estimator_under_score_train= model_performance_classification_sklearn(gb_under,X_train_un, y_train_un,)
gb_estimator_under_score_train

In [ ]:
gb_estimator_under_score_val= model_performance_classification_sklearn(gb_under,X_val_d, y_val,)
gb_estimator_under_score_val

In [ ]:
gb_estimator_under_cm = confusion_matrix_sklearn(gb_under, X_val_d, y_val)
gb_estimator_under_cm

* The abc undersampled model had good generalizability with the f1 score. However, it still performed poorly. The rest were overfit

## Hyperparameter Tuning

**Best practices for hyperparameter tuning in AdaBoost:**

`n_estimators`:

- Start with a specific number (50 is used in general) and increase in steps: 50, 75, 85, 100

- Use fewer estimators (e.g., 50 to 100) if using complex base learners (like deeper decision trees)

- Use more estimators (e.g., 100 to 150) when learning rate is low (e.g., 0.1 or lower)

- Avoid very high values unless performance keeps improving on validation

`learning_rate`:

- Common values to try: 1.0, 0.5, 0.1, 0.01

- Use 1.0 for faster training, suitable for fewer estimators

- Use 0.1 or 0.01 when using more estimators to improve generalization

- Avoid very small values (< 0.01) unless you plan to use many estimators (e.g., >500) and have sufficient data


---

In [ ]:
# ----- labels & scorer (minority = positive) -----
y_train_1d = np.asarray(y_train).ravel()
pos_label = pd.Series(y_train_1d).value_counts().idxmin()
f1_scorer = make_scorer(f1_score, pos_label=pos_label, zero_division=0)

# ----- AdaBoost + base tree (depth fixed to 4 per your spec) -----
base_dt = DecisionTreeClassifier(max_depth=4, random_state=1)

# Handle old/new sklearn API
use_new = "estimator" in AdaBoostClassifier().get_params()
if use_new:
    abc_tuned = AdaBoostClassifier(estimator=base_dt, random_state=1)
    param_grid = {
        "n_estimators": [50, 75, 85, 100],
        "learning_rate": [1.0, 0.5],
        # if you want to keep depth variable, add: "estimator__max_depth": [2,3,4]
    }
else:
    abc_tuned = AdaBoostClassifier(base_estimator=base_dt, random_state=1)
    param_grid = {
        "n_estimators": [50, 75, 85, 100],
        "learning_rate": [1.0, 0.5],
        # "base_estimator__max_depth": [2,3,4]
    }

# ----- randomized search (use cv=3 to keep it quick) -----
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
search = RandomizedSearchCV(
    estimator=abc_tuned,
    param_distributions=param_grid,
    n_iter=8,                  # covers all combos here
    scoring=f1_scorer,
    cv=cv,
    refit=True,
    n_jobs=-1,                 # set to 1 if RAM is tight
    verbose=1,
    random_state=1,
    return_train_score=False,
    error_score="raise"
)

# >>> use your ONE-HOT encoded training features <<<
search.fit(X_train_d, y_train_1d)

print("Best params:", search.best_params_)
print("Best CV F1: {:.4f}".format(search.best_score_))

best_ada = search.best_estimator_


In [ ]:
abc_tuned_score_train=model_performance_classification_sklearn(best_ada,X_train_d, y_train,)
abc_tuned_score_train

In [ ]:
abc_tuned_score_val= model_performance_classification_sklearn(best_ada,X_val_d, y_val,)
abc_tuned_score_val

In [ ]:
abc_tuned_cm = confusion_matrix_sklearn(best_ada, X_val_d, y_val)
abc_tuned_cm

Use randomized search cv to find ideal parameters

**Best practices for hyperparameter tuning in Random Forest:**


`n_estimators`:

* Start with a specific number (50 is used in general) and increase in steps: 50, 75, 100, 125
* Higher values generally improve performance but increase training time
* Use 100-150 for large datasets or when variance is high


`min_samples_leaf`:

* Try values like: 1, 2, 4, 5, 10
* Higher values reduce model complexity and help prevent overfitting
* Use 1–2 for low-bias models, higher (like 5 or 10) for more regularized models
* Works well in noisy datasets to smooth predictions


`max_features`:

* Try values: `"sqrt"` (default for classification), `"log2"`, `None`, or float values (e.g., `0.3`, `0.5`)
* `"sqrt"` balances between diversity and performance for classification tasks
* Lower values (e.g., `0.3`) increase tree diversity, reducing overfitting
* Higher values (closer to `1.0`) may capture more interactions but risk overfitting


`max_samples` (for bootstrap sampling):

* Try float values between `0.5` to `1.0` or fixed integers
* Use `0.6–0.9` to introduce randomness and reduce overfitting
* Smaller values increase diversity between trees, improving generalization

---

In [ ]:
%%time
# --- labels & scorer (minority = positive) ---
y_train_1d = np.asarray(y_train).ravel()
pos_label = pd.Series(y_train_1d).value_counts().idxmin()
f1_scorer = make_scorer(f1_score, pos_label=pos_label, zero_division=0)

# --- model ---
rf_tuned = RandomForestClassifier(
    random_state=1,
    n_jobs=-1,          # parallelize tree building     # required if you tune max_samples
)

# --- search space ---
param_distributions = {
    "n_estimators": [50, 75, 100],
    "min_samples_leaf": [5, 10],
    "max_features": ["sqrt"],        # use sqrt for speed on wide data
    "max_samples": [0.5, 0.7, 1.0],  # fraction of rows per tree (needs bootstrap=True)
    "class_weight": ["balanced", "balanced_subsample"],
}

# --- CV + search ---
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)  # keep 2 for speed (your choice)
search = RandomizedSearchCV(
    estimator=rf_tuned,
    param_distributions=param_distributions,
    n_iter=12,                    # small, fast sweep (adjust if you like)
    scoring=f1_scorer,
    cv=cv,
    n_jobs=-1,                    # set to 1 if RAM is tight
    verbose=2,
    random_state=1,
    return_train_score=False,
    refit=True,
    error_score="raise"
)

# >>> use your ONE-HOT encoded training matrix <<<
search.fit(X_train_d, y_train_1d)

print("Best params:", search.best_params_)
print("Best CV F1: {:.4f}".format(search.best_score_))

rf_tuned = search.best_estimator_   # already fitted since refit=True

# If you want to evaluate:
# from sklearn.metrics import classification_report
# print(classification_report(y_val, rf_tuned.predict(X_val_d), digits=3, zero_division=0))



Use randomized search cv to find ideal parameters

In [ ]:
rf_tuned_score_train=model_performance_classification_sklearn(rf_tuned,X_train_d, y_train,)
rf_tuned_score_train

In [ ]:
rf_tuned_score_val= model_performance_classification_sklearn(rf_tuned,X_val_d, y_val,)
rf_tuned_score_val

In [ ]:
rf_tuned_cm = confusion_matrix_sklearn(rf_tuned, X_val_d, y_val)
rf_tuned_cm

**Best practices for hyperparameter tuning in Gradient Boosting:**

`n_estimators`:

* Start with 100 (default) and increase: 100, 200, 300, 500
* Typically, higher values lead to better performance, but they also increase training time
* Use 200–500 for larger datasets or complex problems
* Monitor validation performance to avoid overfitting, as too many estimators can degrade generalization


`learning_rate`:

* Common values to try: 0.1, 0.05, 0.01, 0.005
* Use lower values (e.g., 0.01 or 0.005) if you are using many estimators (e.g., > 200)
* Higher learning rates (e.g., 0.1) can be used with fewer estimators for faster convergence
* Always balance the learning rate with `n_estimators` to prevent overfitting or underfitting


`subsample`:

* Common values: 0.7, 0.8, 0.9, 1.0
* Use a value between `0.7` and `0.9` for improved generalization by introducing randomness
* `1.0` uses the full dataset for each boosting round, potentially leading to overfitting
* Reducing `subsample` can help reduce overfitting, especially in smaller datasets


`max_features`:

* Common values: `"sqrt"`, `"log2"`, or float (e.g., `0.3`, `0.5`)
* `"sqrt"` (default) works well for classification tasks
* Lower values (e.g., `0.3`) help reduce overfitting by limiting the number of features considered at each split

---

In [ ]:
# --- labels & scorer (minority = positive) ---
y_train_1d = np.asarray(y_train).ravel()
pos_label = pd.Series(y_train_1d).value_counts().idxmin()
f1_scorer = make_scorer(f1_score, pos_label=pos_label, zero_division=0)

# --- model (kept your init; note: init=None is usually faster/standard) ---
gbc_tuned = GradientBoostingClassifier(
    init=AdaBoostClassifier(random_state=1),
    random_state=1
)

# --- search space (your settings) ---
param_distributions = {
    "n_estimators": [100, 200, 300, 500],
    "subsample":    [0.8, 0.9, 1.0],
    "max_features": [0.3, 0.5],          # fraction of features per split
    "learning_rate":[0.1, 0.05, 0.01, 0.005],
}

# --- CV + randomized search ---
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
search = RandomizedSearchCV(
    estimator=gbc_tuned,
    param_distributions=param_distributions,
    n_iter=20,                  # covers a good portion of the 96 combos
    scoring=f1_scorer,
    cv=cv,
    n_jobs=-1,                  # parallelize across candidates/folds
    verbose=1,
    random_state=1,
    return_train_score=False,
    error_score="raise",
    refit=True
)

# >>> use your ONE-HOT encoded training features <<<
search.fit(X_train_d, y_train_1d)

print("Best params:", search.best_params_)
print("Best CV F1: {:.4f}".format(search.best_score_))

gbc_tuned = search.best_estimator_   # already fit (refit=True)

In [ ]:
gbc_tuned_score_train=model_performance_classification_sklearn(gbc_tuned,X_train_d, y_train,)
gbc_tuned_score_train

In [ ]:
gbc_tuned_score_val= model_performance_classification_sklearn(gbc_tuned,X_val_d, y_val,)
gbc_tuned_score_val

In [ ]:
gbc_tuned_cm = confusion_matrix_sklearn(gbc_tuned, X_val_d, y_val)
gbc_tuned_cm

**Best practices for hyperparameter tuning in XGBoost:**

`n_estimators`:

* Start with 50 and increase in steps: 50,75,100,125.
* Use more estimators (e.g., 150-250) when using lower learning rates
* Monitor validation performance
* High values improve learning but increase training time

`subsample`:

* Common values: 0.5, 0.7, 0.8, 1.0
* Use `0.7–0.9` to introduce randomness and reduce overfitting
* `1.0` uses the full dataset in each boosting round; may overfit on small datasets
* Values < 0.5 are rarely useful unless dataset is very large

`gamma`:

* Try values: 0 (default), 1, 3, 5, 8
* Controls minimum loss reduction needed for a split
* Higher values make the algorithm more conservative (i.e., fewer splits)
* Use values > 0 to regularize and reduce overfitting, especially on noisy data


`colsample_bytree`:

* Try values: 0.3, 0.5, 0.7, 1.0
* Fraction of features sampled per tree
* Lower values (e.g., 0.3 or 0.5) increase randomness and improve generalization
* Use `1.0` when you want all features considered for every tree


`colsample_bylevel`:

* Try values: 0.3, 0.5, 0.7, 1.0
* Fraction of features sampled at each tree level (i.e., per split depth)
* Lower values help in regularization and reducing overfitting
* Often used in combination with `colsample_bytree` for fine control over feature sampling

---

In [ ]:
# --- labels & scorer (minority = positive) ---
y_train_1d = np.asarray(y_train).ravel()
pos_label = pd.Series(y_train_1d).value_counts().idxmin()
f1_scorer = make_scorer(f1_score, pos_label=pos_label, zero_division=0)

# --- model (use hist for speed) ---
xgb_tuned = XGBClassifier(
    random_state=1,
    eval_metric='logloss',
    tree_method='hist',
    n_jobs=-1,
    verbosity=0
)

# --- search space (fixed n_estimators list; your np.arange was invalid) ---
param_distributions = {
    "n_estimators": [50, 75, 100, 125],
    "subsample": [0.5, 0.7, 0.8, 1.0],
    "gamma": [1, 3, 5, 8],
    "colsample_bytree": [0.3, 0.5, 0.7, 1.0],
    "colsample_bylevel": [0.3, 0.5, 0.7, 1.0],
    "max_depth": [3, 4, 5, 6],
    "min_child_weight": [1, 3, 5],
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

search = RandomizedSearchCV(
    estimator=xgb_tuned,
    param_distributions=param_distributions,
    n_iter=20,                 # adjust for time budget
    scoring=f1_scorer,
    cv=cv,
    n_jobs=-1,                 # use all cores; set to 1 if RAM is tight
    verbose=1,
    random_state=1,
    return_train_score=False,
    error_score="raise",
    refit=True
)

# >>> use your ONE-HOT encoded training features <<<
search.fit(X_train_d, y_train_1d)

print("Best params:", search.best_params_)
print("Best CV F1: {:.4f}".format(search.best_score_))

xgb_tuned = search.best_estimator_   # already fit since refit=True

In [ ]:
xgb_tuned_score_train=model_performance_classification_sklearn(xgb_tuned,X_train_d, y_train,)
xgb_tuned_score_train

In [ ]:
xgb_tuned_score_val= model_performance_classification_sklearn(xgb_tuned,X_val_d, y_val,)
xgb_tuned_score_val

In [ ]:
xgb_tuned_cm = confusion_matrix_sklearn(xgb_tuned, X_val_d, y_val)
xgb_tuned_cm

## Model Performance Summary and Final Model Selection

In [ ]:
# positive = minority class
pos_label = pd.Series(np.asarray(y_train).ravel()).value_counts().idxmin()

rows = []
for name, model in models_dict.items():
    # ensure the model is already fitted
    y_tr_pred = model.predict(X_train_d)
    y_te_pred = model.predict(X_test_d)

    acc_tr = accuracy_score(y_train, y_tr_pred)
    acc_te = accuracy_score(y_test,  y_te_pred)

    rec_tr = recall_score(y_train, y_tr_pred, pos_label=pos_label, zero_division=0)
    rec_te = recall_score(y_test,  y_te_pred, pos_label=pos_label, zero_division=0)

    prec_tr = precision_score(y_train, y_tr_pred, pos_label=pos_label, zero_division=0)
    prec_te = precision_score(y_test,  y_te_pred, pos_label=pos_label, zero_division=0)

    f1_tr = f1_score(y_train, y_tr_pred, pos_label=pos_label, zero_division=0)
    f1_te = f1_score(y_test,  y_te_pred, pos_label=pos_label, zero_division=0)

    rows.append({
        "model": name,
        "acc_train": round(acc_tr, 2),
        "acc_test":  round(acc_te, 2),
        "recall_train": round(rec_tr, 2),
        "recall_test":  round(rec_te, 2),
        "precision_train": round(prec_tr, 2),
        "precision_test":  round(prec_te, 2),
        "f1_train": round(f1_tr, 2),
        "f1_test":  round(f1_te, 2),
    })

results_df = pd.DataFrame(rows).set_index("model")
print(results_df)



compare the tuned models with the original models and oversampled data models (ignore undersampled)

* None of these models did particularly well on the f1 score.
* gbc_tuned and xgb_tuned did very well on precision and had a 75% accuracy score.
* I would choose the xgb_tuned model for generalizability in the f1 score and the higher accuracy result.

In [ ]:
from sklearn.inspection import permutation_importance


# 1) Make sure these are the exact features used to train xgb_tuned
feature_names = list(X_train_d.columns)

def xgb_importances_df(model, feature_names, importance_type="gain"):
    booster = model.get_booster()

    # Try the requested type first; if empty, try alternates
    try_types = [importance_type, "total_gain", "weight", "cover", "total_cover"]
    score = {}
    used_type = None
    for t in try_types:
        score = booster.get_score(importance_type=t)
        if score:
            used_type = t
            break

    # Map keys (either actual names or "fN") to indices
    name_to_idx = {n: i for i, n in enumerate(feature_names)}
    imp = np.zeros(len(feature_names), dtype=float)

    for k, v in score.items():
        if k in name_to_idx:
            imp[name_to_idx[k]] = float(v)
        elif k.startswith("f") and k[1:].isdigit():
            idx = int(k[1:])
            if 0 <= idx < len(imp):
                imp[idx] = float(v)
        # else: key we don't recognize (rare)

    df = pd.DataFrame({"feature": feature_names, "importance": imp})
    df = df.sort_values("importance", ascending=False).reset_index(drop=True)
    return df, (used_type or importance_type)

imp_df, used_type = xgb_importances_df(xgb_tuned, feature_names, importance_type="gain")
print(f"Used importance_type: {used_type}")

# If everything is zero (e.g., model super simple, or keys didn’t map), fall back to permutation importance
if imp_df["importance"].sum() == 0:
    print("All zero importances from Booster; computing permutation importance on validation set...")
    pos_label = pd.Series(np.asarray(y_train).ravel()).value_counts().idxmin()
    scorer = make_scorer(f1_score, pos_label=pos_label, zero_division=0)

    r = permutation_importance(
        xgb_tuned, X_val_d, y_val,
        scoring=scorer, n_repeats=5, random_state=1, n_jobs=-1
    )
    imp_df = pd.DataFrame(
        {"feature": feature_names, "importance": r.importances_mean}
    ).sort_values("importance", ascending=False).reset_index(drop=True)
    used_type = "permutation(F1)"

# Plot top-N
top_n = 30
top = imp_df.head(top_n).iloc[::-1]  # reverse for barh
plt.figure(figsize=(8, 10))
plt.barh(top["feature"], top["importance"])
plt.title(f"XGBoost Feature Importance — Top {top_n} ({used_type})")




In [ ]:
%%time
# Predict on the one-hot encoded TEST set (same columns/order as training)
applicant_details = X_test_d.reindex(columns=X_train_d.columns, fill_value=0)

# For a single applicant, keep 2D shape:
# applicant_details = X_test_d.reindex(columns=X_train_d.columns, fill_value=0).iloc[[0]]

print(xgb_tuned.predict(applicant_details))
print(xgb_tuned.predict_proba(applicant_details)[:, 1])



## Actionable Insights and Recommendations

* Use xgb_tuned for predictive capabilities.
* It would be best to expend further time exploring feature space.
* Time and resources were limited to the expoloration and training of certain models. Therefore, one should consider the relevance of pursuing further time and expanded feature space for more precise model development.

<font size=6 color='blue'>Power Ahead</font>
___